# Play Type Counts

In [1]:
import pandas as pd

## Play type classification function

In [2]:
def play_type(playOutcome):
    play = str(playOutcome).lower()
    if "pass" in play:
        return "Pass"
    elif "rush" in play or "run" in play or "ran" in play:
        return "Run"
    elif "punt" in play:
        return "Punt"
    elif "field goal" in play:
        return "Field Goal"
    elif "intercepted" in play or "fumble" in play or "turnover" in play:
        return "Turnover"
    elif "touchdown" in play:
        return "Touchdown"
    elif "timeout" in play:
        return "Timeout"
    else:
        return "Other"

## Play count for top 10 and bottom 10 teams function

In [3]:
def play_count(scoresURL, playsURL):
    # Import data
    scores = pd.read_csv(scoresURL)
    plays = pd.read_csv(playsURL)
    plays["ShortName"] = plays["TeamWithPossession"].apply(lambda x: str(x).split()[-1])

    # Get top 10 and bottom 10 teams
    top10 = scores.head(10)["Team"].tolist()
    bottom10 = scores.tail(10)["Team"].tolist()

    # Get key columns and apply function
    filteredPlays = plays[["Season", "ShortName", "PlayOutcome", "PlayDescription"]].copy()
    filteredPlays["PlayType"] = filteredPlays["PlayOutcome"].apply(play_type)

    # Filter plays by top 10 and bottom 10
    topPlays = filteredPlays[filteredPlays["ShortName"].isin(top10)]
    bottomPlays = filteredPlays[filteredPlays["ShortName"].isin(bottom10)]

    # Top 10 and bottom 10 play counts
    topCount = topPlays.groupby(["ShortName", "PlayType"]).size().unstack(fill_value=0)
    bottomCount = bottomPlays.groupby(["ShortName", "PlayType"]).size().unstack(fill_value=0)

    return topCount, bottomCount

## 2017 Data

In [4]:
top2017, bottom2017 = play_count("2017_scores.csv", "2017_plays.csv")

In [5]:
top2017

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
Chiefs,53,312,653,87,497,72,52,10
Eagles,49,328,773,96,648,85,70,22
Jaguars,46,290,686,121,722,70,57,12
Patriots,54,348,813,81,606,79,78,13
Rams,54,297,654,90,542,68,56,16
Ravens,46,243,634,109,584,89,46,10
Saints,49,280,706,96,568,88,60,16
Seahawks,40,320,621,103,527,61,51,8
Steelers,45,289,712,89,534,78,57,14


In [6]:
bottom2017

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
Bears,26,292,562,108,520,55,35,18
Browns,25,325,667,104,467,78,39,15
Cardinals,49,321,700,113,535,75,43,12
Colts,40,293,585,111,517,72,35,10
Dolphins,26,310,665,107,443,60,44,16
Giants,38,278,716,111,486,56,32,17
Jets,38,290,600,124,518,74,43,10
Raiders,26,291,643,85,430,66,46,20
Texans,29,333,593,112,511,49,45,16


## 2018 Data

In [7]:
top2018, bottom2018 = play_count("2018_scores.csv", "2018_plays.csv")

In [8]:
top2018

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
Bears,45,328,682,87,589,112,59,17
Chargers,41,324,644,90,533,86,66,8
Chiefs,38,363,730,65,490,91,84,10
Colts,40,338,798,82,547,72,69,20
Patriots,46,307,787,96,673,76,68,11
Rams,56,339,725,82,634,98,67,13
Ravens,58,317,684,94,708,95,52,15
Saints,43,292,641,60,611,81,75,13
Steelers,30,303,734,83,430,74,67,12


In [9]:
bottom2018

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
49ers,40,329,612,85,508,72,46,17
Bills,33,334,579,103,554,64,46,11
Buccaneers,35,340,711,71,471,64,64,13
Cardinals,25,287,558,121,433,52,35,17
Dolphins,32,305,570,104,439,67,45,16
Falcons,30,299,694,84,420,56,54,18
Jets,43,288,587,104,489,67,39,17
Lions,38,287,690,93,479,50,46,13
Raiders,40,312,632,92,468,76,40,21


## 2019 Data

In [10]:
top2019, bottom2019 = play_count("2019_scores.csv", "2019_plays.csv")

In [11]:
top2019

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
49ers,51,337,592,74,693,109,74,15
Bills,40,292,643,99,583,83,48,18
Chiefs,45,360,774,74,503,69,76,18
Cowboys,48,279,717,75,514,58,54,15
Packers,26,341,698,106,529,89,62,16
Patriots,44,258,735,98,576,68,59,14
Ravens,44,300,566,56,723,55,68,10
Saints,50,295,694,80,513,67,59,10
Titans,24,368,589,116,618,58,67,17


In [12]:
bottom2019

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
Bengals,39,313,739,91,444,57,42,23
Cardinals,41,319,657,89,452,78,49,11
Dolphins,40,343,684,87,444,64,42,17
Giants,22,317,699,85,437,46,59,21
Jaguars,38,296,685,101,462,41,33,23
Jets,37,304,610,107,475,60,36,13
Lions,36,339,637,104,481,51,44,10
Panthers,40,317,715,94,445,69,49,24
Raiders,33,256,629,90,539,76,47,15


## 2020 Data

In [13]:
top2020, bottom2020 = play_count("2020_scores.csv", "2020_plays.csv")

In [14]:
top2020

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
Bills,43,288,656,51,443,66,62,12
Buccaneers,40,289,693,67,468,79,74,6
Chiefs,35,273,693,56,449,71,60,11
Colts,39,235,559,51,462,57,50,9
Packers,20,253,548,50,474,65,74,9
Rams,34,230,602,78,508,74,47,13
Ravens,35,242,425,58,589,67,57,10
Saints,31,252,551,64,515,71,63,17
Seahawks,26,279,532,69,419,70,59,9


In [15]:
bottom2020

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
Bengals,34,259,547,71,391,60,35,11
Broncos,35,247,508,65,423,59,37,14
Cowboys,41,233,595,49,407,63,43,16
Eagles,19,300,553,73,378,53,40,16
Giants,32,233,491,63,380,35,26,16
Jaguars,25,236,571,65,324,35,34,18
Jets,28,231,466,82,392,50,28,10
Lions,28,269,543,59,341,52,47,10
Panthers,36,238,514,42,380,63,38,15


## 2021 Data

In [16]:
top2021, bottom2021 = play_count("2021_scores.csv", "2021_plays.csv")

In [17]:
top2021

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
49ers,41,325,612,89,661,108,60,23
Bengals,56,367,726,93,589,91,68,12
Bills,38,298,770,65,551,77,77,13
Buccaneers,39,302,873,87,453,72,75,17
Chiefs,41,349,842,61,551,89,78,17
Colts,45,277,585,70,565,80,55,17
Cowboys,43,339,788,88,572,64,66,13
Patriots,49,287,615,62,564,70,64,10
Rams,51,319,781,79,596,94,69,11


In [18]:
bottom2021

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
Bears,32,303,584,78,515,64,41,21
Falcons,33,293,592,75,447,64,38,19
Giants,34,291,640,91,465,51,32,19
Jaguars,33,289,705,83,427,66,38,16
Jets,33,303,640,83,460,73,43,15
Lions,34,314,638,76,479,76,43,17
Panthers,39,291,653,86,496,68,39,22
Raiders,51,321,725,79,482,77,46,16
Steelers,48,304,785,100,534,73,44,17


## 2022 Data

In [19]:
top2022, bottom2022 = play_count("2022_scores.csv", "2022_plays.csv")

In [20]:
top2022

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
49ers,44,305,640,80,643,96,62,23
Bengals,44,334,770,80,509,101,63,11
Bills,36,301,688,60,515,80,67,18
Chiefs,42,306,787,79,539,95,75,11
Cowboys,40,306,655,89,653,72,68,11
Dolphins,44,298,662,74,456,82,54,11
Eagles,30,348,670,78,703,104,82,19
Lions,37,281,674,59,529,74,61,15
Raiders,46,301,653,77,529,65,52,9


In [21]:
bottom2022

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
Bears,32,290,419,83,622,55,48,20
Broncos,43,312,649,104,492,75,36,11
Buccaneers,43,264,860,102,465,73,38,17
Cardinals,38,316,735,80,479,87,43,14
Colts,44,299,642,84,486,58,40,22
Rams,32,289,565,83,472,65,42,12
Seahawks,45,334,665,85,503,76,52,19
Texans,36,277,602,101,466,43,43,14
Titans,29,293,514,107,542,70,40,14


## 2023 Data

In [22]:
top2023, bottom2023 = play_count("2023_scores.csv", "2023_plays.csv")

In [23]:
top2023

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
49ers,38,331,633,74,616,90,75,17
Bills,37,299,691,66,630,79,68,13
Chiefs,50,319,832,84,576,82,63,15
Cowboys,42,318,722,55,552,64,65,11
Dolphins,31,316,635,62,520,69,71,22
Jaguars,40,299,663,66,517,65,51,27
Lions,29,325,764,85,612,88,75,13
Packers,40,305,673,69,574,86,61,15
Ravens,44,315,583,84,639,78,70,22


In [24]:
bottom2023

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
Cardinals,39,309,629,77,534,70,43,13
Commanders,28,332,689,84,413,67,52,17
Falcons,39,277,586,89,566,68,34,17
Giants,36,319,576,106,509,57,36,18
Jets,48,304,685,112,463,61,28,28
Panthers,33,303,653,95,494,46,29,24
Patriots,26,265,584,114,460,55,36,16
Rams,50,308,662,81,534,89,54,8
Titans,37,332,536,88,525,65,40,10


## 2024 Data

In [25]:
top2024, bottom2024 = play_count("2024_scores.csv", "2024_plays.csv")

In [26]:
top2024

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
Bills,42,342,634,73,646,89,77,8
Broncos,40,284,630,90,548,80,55,13
Buccaneers,38,320,622,65,556,61,66,12
Chargers,50,286,597,94,522,73,44,8
Commanders,58,360,706,68,670,87,67,16
Eagles,49,333,583,76,763,127,71,18
Lions,37,302,632,60,616,83,79,14
Packers,48,291,543,64,614,74,59,18
Ravens,34,312,532,75,680,72,75,15


In [27]:
bottom2024

PlayType,Field Goal,Other,Pass,Punt,Run,Timeout,Touchdown,Turnover
ShortName,,,,,,,,
Browns,31,334,728,100,439,62,38,20
Cowboys,56,311,704,68,509,66,38,18
Falcons,48,289,608,62,553,66,44,12
Giants,31,301,648,93,473,42,39,18
Jaguars,36,263,588,77,501,58,40,18
Panthers,30,316,586,89,454,61,47,15
Patriots,39,307,597,88,502,57,36,20
Raiders,46,299,710,82,426,58,38,17
Saints,37,293,610,91,508,72,44,7
